In [ ]:
# ============================================================================
# CSRNET TRAINING - SIMPLE WORKING VERSION
# Part A Dataset Only - Guaranteed to work
# ============================================================================

import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

# ==================== CELL 1: VERIFY PREPROCESSING ===========================

BASE_PATH = "/Users/akshara/Documents/info/ShanghaiTech/preprocessed"
TRAIN_IMAGES = os.path.join(BASE_PATH, "train_images")
TRAIN_DENSITY = os.path.join(BASE_PATH, "train_density")
TEST_IMAGES = os.path.join(BASE_PATH, "test_images")
TEST_DENSITY = os.path.join(BASE_PATH, "test_density")

print("🔍 VERIFYING PREPROCESSING...")
train_img_files = sorted([f for f in os.listdir(TRAIN_IMAGES) if f.endswith('.npy')])
train_den_files = sorted([f for f in os.listdir(TRAIN_DENSITY) if f.endswith('.npy')])

print(f"✅ Train Images: {len(train_img_files)}")
print(f"✅ Train Density: {len(train_den_files)}")

if len(train_img_files) > 0:
    sample_img = np.load(os.path.join(TRAIN_IMAGES, train_img_files[0]))
    sample_den = np.load(os.path.join(TRAIN_DENSITY, train_den_files[0]))
    print(f"✅ Image shape: {sample_img.shape}")
    print(f"✅ Density shape: {sample_den.shape}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Device: {device}\n")

# ==================== CELL 2: SETUP ==========================================

torch.manual_seed(42)
np.random.seed(42)

# ==================== CELL 3: DATASET CLASS ==================================

class CrowdDataset(Dataset):
    """Dataset class"""
    
    def __init__(self, images_dir, density_dir):
        self.images_dir = images_dir
        self.density_dir = density_dir
        self.image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.npy')])
        self.density_files = sorted([f for f in os.listdir(density_dir) if f.endswith('.npy')])
        print(f"📂 Loaded {len(self.image_files)} samples")
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        image = np.load(os.path.join(self.images_dir, self.image_files[idx]))
        density = np.load(os.path.join(self.density_dir, self.density_files[idx]))
        
        image = torch.FloatTensor(image).permute(2, 0, 1)
        density = torch.FloatTensor(density).unsqueeze(0)
        
        return image, density

train_dataset = CrowdDataset(TRAIN_IMAGES, TRAIN_DENSITY)
test_dataset = CrowdDataset(TEST_IMAGES, TEST_DENSITY)

# ==================== CELL 4: SIMPLE WORKING CSRNET ==========================

class SimpleCrowdNet(nn.Module):
    """
    Simplified Crowd Counting Network
    - Fewer downsampling layers
    - Simple upsampling to restore dimensions
    - Guaranteed to output 256×256
    """
    
    def __init__(self):
        super(SimpleCrowdNet, self).__init__()
        
        # ========== FRONTEND: Feature Extraction (2 MaxPool = 256→64) ==========
        self.conv1 = self._make_conv_block(3, 64, 2)      # 256
        self.pool1 = nn.MaxPool2d(2, 2)                   # 128
        
        self.conv2 = self._make_conv_block(64, 128, 2)    # 128
        self.pool2 = nn.MaxPool2d(2, 2)                   # 64
        
        self.conv3 = self._make_conv_block(128, 256, 2)   # 64
        self.pool3 = nn.MaxPool2d(2, 2)                   # 32
        
        self.conv4 = self._make_conv_block(256, 512, 2)   # 32
        
        # ========== BACKEND: Feature Refinement (Dilated Conv) ==========
        self.conv5 = self._make_conv_block(512, 512, 1)   # 32
        self.conv6 = self._make_conv_block(512, 256, 1)   # 32
        
        # ========== DECODER: Restore Dimensions (32 → 256) ==========
        # Upsampling path: 32 → 64 → 128 → 256
        self.upsample1 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            nn.Conv2d(256, 128, 3, padding=1),
            nn.ReLU(inplace=True)
        )  # 32 → 64
        
        self.upsample2 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            nn.Conv2d(128, 64, 3, padding=1),
            nn.ReLU(inplace=True)
        )  # 64 → 128
        
        self.upsample3 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            nn.Conv2d(64, 32, 3, padding=1),
            nn.ReLU(inplace=True)
        )  # 128 → 256
        
        # ========== OUTPUT: Density Map ==========
        self.output = nn.Conv2d(32, 1, 1)
        
        self._init_weights()
    
    def _make_conv_block(self, in_ch, out_ch, num_conv):
        """Create a block of convolutional layers"""
        layers = []
        for i in range(num_conv):
            layers.append(nn.Conv2d(in_ch if i == 0 else out_ch, out_ch, 3, padding=1))
            layers.append(nn.ReLU(inplace=True))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        # Frontend - Downsampling
        x = self.conv1(x)           # (B, 64, 256, 256)
        x = self.pool1(x)           # (B, 64, 128, 128)
        
        x = self.conv2(x)           # (B, 128, 128, 128)
        x = self.pool2(x)           # (B, 128, 64, 64)
        
        x = self.conv3(x)           # (B, 256, 64, 64)
        x = self.pool3(x)           # (B, 256, 32, 32)
        
        x = self.conv4(x)           # (B, 512, 32, 32)
        
        # Backend - Feature refinement
        x = self.conv5(x)           # (B, 512, 32, 32)
        x = self.conv6(x)           # (B, 256, 32, 32)
        
        # Decoder - Upsampling back to 256×256
        x = self.upsample1(x)       # (B, 128, 64, 64)
        x = self.upsample2(x)       # (B, 64, 128, 128)
        x = self.upsample3(x)       # (B, 32, 256, 256)
        
        # Output density map
        x = self.output(x)          # (B, 1, 256, 256)
        
        return x
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

# ==================== CELL 5: TEST MODEL DIMENSIONS ===========================

print("\n" + "="*60)
print("🧠 INITIALIZING MODEL")
print("="*60)

model = SimpleCrowdNet().to(device)

# Test dimensions
print("\n✅ Testing model dimensions...")
test_input = torch.randn(1, 3, 256, 256).to(device)
test_output = model(test_input)

print(f"   Input shape:    {tuple(test_input.shape)}")
print(f"   Output shape:   {tuple(test_output.shape)}")
print(f"   Expected:       (1, 1, 256, 256)")

if test_output.shape == torch.Size([1, 1, 256, 256]):
    print("   ✅ DIMENSIONS CORRECT!")
else:
    print(f"   ❌ DIMENSIONS WRONG: {tuple(test_output.shape)}")
    raise RuntimeError(f"Wrong output shape: {tuple(test_output.shape)}")

total_params = sum(p.numel() for p in model.parameters())
print(f"\n📊 Model Parameters: {total_params:,}")

# ==================== CELL 6: TRAINING SETUP =================================

criterion = nn.MSELoss()
learning_rate = 1e-5
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

BATCH_SIZE = 8
NUM_EPOCHS = 150

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print(f"\n⚙️ Optimizer: Adam (lr={learning_rate})")
print(f"📦 Batch Size: {BATCH_SIZE}")
print(f"🔄 Train Batches: {len(train_loader)}")
print(f"⏱️ Epochs: {NUM_EPOCHS}")

batch_losses = []
epoch_losses = []

# ==================== CELL 7: TRAINING LOOP ==================================

def train_one_epoch(epoch):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    num_batches = len(train_loader)
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
    
    for batch_idx, (images, density_maps) in enumerate(pbar):
        images = images.to(device)
        density_maps = density_maps.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, density_maps)
        loss.backward()
        optimizer.step()
        
        batch_loss = loss.item()
        batch_losses.append(batch_loss)
        running_loss += batch_loss
        
        pbar.set_postfix({'Loss': f'{batch_loss:.6f}'})
        
        if np.isnan(batch_loss) or np.isinf(batch_loss):
            print(f"\n⚠️ NaN detected!")
            return None
    
    epoch_loss = running_loss / num_batches
    epoch_losses.append(epoch_loss)
    return epoch_loss

def evaluate():
    """Evaluate on test set"""
    model.eval()
    running_loss = 0.0
    
    with torch.no_grad():
        for images, density_maps in tqdm(test_loader, desc="Testing"):
            images = images.to(device)
            density_maps = density_maps.to(device)
            outputs = model(images)
            loss = criterion(outputs, density_maps)
            running_loss += loss.item()
    
    return running_loss / len(test_loader)

# ==================== START TRAINING =========================================

print("\n" + "="*60)
print("🔥 STARTING TRAINING")
print("="*60)

best_loss = float('inf')

try:
    for epoch in range(NUM_EPOCHS):
        train_loss = train_one_epoch(epoch)
        
        if train_loss is None:
            break
        
        # Evaluate every 5 epochs
        if (epoch + 1) % 5 == 0:
            test_loss = evaluate()
            print(f"\n🧪 Epoch {epoch+1} - Test Loss: {test_loss:.6f}")
            
            if test_loss < best_loss:
                best_loss = test_loss
                torch.save(model.state_dict(), 'csrnet_best_weights.pth')
                print(f"💾 Best model saved!")
        
        print(f"{'='*60}")
        print(f"📊 Epoch {epoch+1}: Train={train_loss:.6f}, Best={best_loss:.6f}")
        print(f"{'='*60}\n")
        
        # Save progress every 10 epochs
        if (epoch + 1) % 10 == 0:
            np.save('batch_losses.npy', np.array(batch_losses))
            np.save('epoch_losses.npy', np.array(epoch_losses))

except KeyboardInterrupt:
    print("\n⚠️ Training interrupted")

# ==================== SAVE & VISUALIZE =======================================

print("\n" + "="*60)
print("💾 SAVING MODEL")
print("="*60)

torch.save(model.state_dict(), 'csrnet_weights.pth')
np.save('batch_losses.npy', np.array(batch_losses))
np.save('epoch_losses.npy', np.array(epoch_losses))

print("✅ Model saved: csrnet_weights.pth")
print("✅ Losses saved: batch_losses.npy, epoch_losses.npy")

# Plot results
if len(epoch_losses) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    axes[0].plot(batch_losses, alpha=0.5, color='blue')
    axes[0].set_title('Batch Loss')
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(epoch_losses, marker='o', color='red')
    axes[1].set_title('Epoch Loss')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('training_progress.png', dpi=150)
    plt.show()
    
    print(f"\n📈 Summary:")
    print(f"   Initial Loss: {epoch_losses[0]:.6f}")
    print(f"   Final Loss:   {epoch_losses[-1]:.6f}")
    print(f"   Best Loss:    {best_loss:.6f}")

print("\n" + "="*60)
print("✅ TRAINING COMPLETE!")
print("="*60)


🔍 VERIFYING PREPROCESSING...
✅ Train Images: 300
✅ Train Density: 300
✅ Image shape: (256, 256, 3)
✅ Density shape: (256, 256)
✅ Device: cpu

📂 Loaded 300 samples
📂 Loaded 182 samples

🧠 INITIALIZING MODEL

✅ Testing model dimensions...
   Input shape:    (1, 3, 256, 256)
   Output shape:   (1, 1, 256, 256)
   Expected:       (1, 1, 256, 256)
   ✅ DIMENSIONS CORRECT!

📊 Model Parameters: 8,612,417

⚙️ Optimizer: Adam (lr=1e-05)
📦 Batch Size: 8
🔄 Train Batches: 38
⏱️ Epochs: 150

🔥 STARTING TRAINING


Epoch 1/150: 100%|███████████████| 38/38 [02:34<00:00,  4.07s/it, Loss=0.000173]


📊 Epoch 1: Train=0.000412, Best=inf



Epoch 2/150: 100%|███████████████| 38/38 [02:13<00:00,  3.52s/it, Loss=0.000175]


📊 Epoch 2: Train=0.000405, Best=inf



...

Testing: 100%|██████████████████████████████████| 23/23 [00:28<00:00,  1.26s/it]



🧪 Epoch 50 - Test Loss: 0.000115
💾 Best model saved!
📊 Epoch 50: Train=0.000195, Best=0.000115



Epoch 51/150: 100%|██████████████| 38/38 [02:30<00:00,  3.96s/it, Loss=0.000060]


📊 Epoch 51: Train=0.000182, Best=0.000115



Epoch 52/150:  16%|██▎            | 6/38 [00:23<02:06,  3.96s/it, Loss=0.000167]